[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gdslab/d2spy/blob/master/tutorial/d2s_geoai_tutorial_01.ipynb)
[![Jupyter Notebook](https://img.shields.io/badge/Open%20in%20JuypterHub%20-%20%233776AB?logo=jupyter&logoColor=%23F37626&labelColor=%23F5F5F5)](https://lab.d2s.org)
[![License: MIT](https://img.shields.io/badge/License-MIT-yellow.svg)](https://opensource.org/licenses/MIT)


# Visualizing building footprint and aerial imageries

This tutorial will demonstrate how to download building footprints from Overture and display them over various aerial imageries such as NAIP, Indiana Orthomosaic images, and Indiana LiDAR data.

This tutorial requires following modules to be installed.

* d2spy: https://py.d2s.org/
* geoai: https://geoai.gishub.org/
* leafmap: https://leafmap.org/

## Import Modules

Importing required modules. 

In [1]:
import leafmap
import d2spy
from pystac_client import Client
from d2spy.extras.utils import clip_by_mask
from d2spy.workspace import Workspace
from geoai.download import (
    download_naip,
    download_overture_buildings,
    extract_building_stats,
)

In [2]:
def bbox_to_geojson(bbox):
    """
    Convert a bounding box to a GeoJSON Polygon feature.
    
    Parameters:
        bbox is a list with the following order.
        min_x (float): Minimum longitude (x coordinate).
        min_y (float): Minimum latitude (y coordinate).
        max_x (float): Maximum longitude (x coordinate).
        max_y (float): Maximum latitude (y coordinate).
    
    Returns:
        dict: A GeoJSON feature representing the bounding box.
    """
    # Create the list of coordinates for the polygon.
    # GeoJSON polygons require the first and last coordinates to be identical.
    min_x = bbox[0]
    min_y = bbox[1]
    max_x = bbox[2]
    max_y = bbox[3]
    
    coordinates = [
        [min_x, min_y],
        [max_x, min_y],
        [max_x, max_y],
        [min_x, max_y],
        [min_x, min_y]
    ]
    
    # Build the GeoJSON object (as a Feature)
    geojson_feature = {
        "type": "Feature",
        "geometry": {
            "type": "Polygon",
            "coordinates": [coordinates]
        },
        "properties": {}
    }
    
    return geojson_feature

## Define bounding box

We will use leafmap to define a geographic extent (longitude and latitude) of our area of interests.

In [3]:
m = leafmap.Map(center=[40.4259, -86.9081], zoom=16)
m.add_basemap("Google Satellite")
m

Map(center=[40.4259, -86.9081], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

Use the drawing tool, you can draw a rectangle on the map. If no rectangle is drawn, the default ROI will be used. 

In [4]:
bbox = m.user_roi_bounds()
if bbox is None:
    bbox = (-86.9193, 40.4241, -86.9102, 40.4314)

## Query Indiana Ortho Imagery

We will use D2S (Data to Science) STAC to find available orthomosaic images around the AOI (Area of Interests) we defined above.

In [5]:
# Connect to STAC API
client = Client.open("https://stac-api.d2s.org")

# Search for items from the NAIP collection
search = client.search(
    max_items=10,
    collections=["naip"],
    bbox=bbox,
    datetime=['2011-01-01', '2020-12-31'],
)

# Print STAC Item ID and STAC Browser URL for search results
stac_browser_base_item_url = "https://stac.d2s.org/collections/naip/items"
items = []
for item in search.items():
    print(f"ID: {item.id}, URL: {stac_browser_base_item_url}/{item.id}")
    items.append(item)

ID: in_m_4008633_se_16_060_20200611, URL: https://stac.d2s.org/collections/naip/items/in_m_4008633_se_16_060_20200611
ID: in_m_4008633_se_16_060_20180713_20181211, URL: https://stac.d2s.org/collections/naip/items/in_m_4008633_se_16_060_20180713_20181211
ID: in_m_4008633_se_16_h_20160903, URL: https://stac.d2s.org/collections/naip/items/in_m_4008633_se_16_h_20160903
ID: in_m_4008633_se_16_1_20141008_20141201, URL: https://stac.d2s.org/collections/naip/items/in_m_4008633_se_16_1_20141008_20141201
ID: in_m_4008633_se_16_1_20120610_20120821, URL: https://stac.d2s.org/collections/naip/items/in_m_4008633_se_16_1_20120610_20120821


Check how many NAIP images are available.

In [6]:
print(f"We found {len(items)} NAIP images over your AOI {bbox}.")

We found 5 NAIP images over your AOI (-86.9193, 40.4241, -86.9102, 40.4314).


Now let's choose the first NAIP image and clip it using the bounding box we created earlier to save it on your local machine.

In [7]:
# Choose the first item
item = items[0]

# Print the name of the asset available in the STAC item
for asset in item.assets:
    print(asset)

#The URL for the NAIP raster is in the "image" asset
naip_url = item.assets["image"].href
print(naip_url)

image
tilejson
thumbnail
rendered_preview
https://naipeuwest.blob.core.windows.net/naip/v002/in/2020/in_060cm_2020/40086/m_4008633_se_16_060_20200611.tif


The NAIP raster is a Cloud Optimized GeoTIFF, allowing us to stream the dataset directly, rather than downloading it in full before clipping it.

In [8]:
# Desired location and file name for the clipped raster
clipped_naip_fn = 'naip_data/clipped_naip.tif'

# Need to turn the bbox into GeoJSON object
bbox_geojson = bbox_to_geojson(bbox)

# Now let's clip it
clip_by_mask(in_raster=naip_url, geojson=bbox_geojson, out_raster=clipped_naip_fn)

Now we can visualize the clipped NAIP image.

In [10]:
m = leafmap.Map()
m.add_raster(clipped_naip_fn, layer_name="NAIP 2020")
m

Map(center=[40.4277505, -86.9147495], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

## Download Building Data

Retrieve building footprint data in GeoJSON format within the bounding box. The `verbose` flag provids detailed output.

In [12]:
# Download buildings
buildings_geojson = download_overture_buildings(
    bbox=bbox,
    output_file="buildings.geojson",
    output_format="geojson",
    data_type="building",
    verbose=True,
)

2025-03-09 10:03:54,275 - INFO - Running command: overturemaps download --bbox -86.9193,40.4241,-86.9102,40.4314 -f geojson --type building --output buildings.geojson
2025-03-09 10:03:54,277 - INFO - Downloading building data for area: -86.9193,40.4241,-86.9102,40.4314
2025-03-09 10:04:41,826 - INFO - Successfully downloaded data to buildings.geojson (0.13 MB)
2025-03-09 10:04:41,855 - INFO - Downloaded 148 features
2025-03-09 10:04:41,855 - INFO - Available attributes: id, version, sources, level, subtype, class, height, names, has_parts, is_underground...


If the building data file is successfully downloaded, extract and display relevant statstics such as area, count, and footprint details.

In [13]:
if buildings_geojson:
    stats = extract_building_stats(buildings_geojson)
    print(stats)

{'total_buildings': 148, 'has_height': 125, 'has_name': 0, 'bbox': [-86.9200103, 40.4240726, -86.9093137, 40.4312994]}


Now you can visualize the downloaded NAIP and building footprint together.

In [14]:
m = leafmap.Map()
m.add_raster(clipped_naip_fn, layer_name="NAIP 2020")
m.add_geojson(buildings_geojson, layer_name="Buildings")
m

Map(center=[40.4277505, -86.9147495], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

## Misalignment between building footprints and NAIP imageries

We can clearly see the misalignment between building footprints and NAIP imageries. There could be many reasons why this could happen, but let's see if this issue is resolved when we use more accurate basemap layers. We will use Indiana Ortho Imageries as a basemap layer to check this.

In [15]:
# Search for items from the Indiana Ortho Imagery Collection
search = client.search(
    max_items=10,
    collections=["ortho"],
    bbox=bbox,
    datetime=['2011-01-01', '2024-12-31'],
)

# Print STAC Item ID and STAC Browser URL for search results
stac_browser_base_item_url = "https://stac.d2s.org/collections/naip/items"
items = []
for item in search.items():
    print(f"ID: {item.id}, URL: {stac_browser_base_item_url}/{item.id}")
    items.append(item)

ID: tippecanoe-2023-ortho, URL: https://stac.d2s.org/collections/naip/items/tippecanoe-2023-ortho
ID: tippecanoe-2018-ortho, URL: https://stac.d2s.org/collections/naip/items/tippecanoe-2018-ortho
ID: flood-imagery-2016-ortho, URL: https://stac.d2s.org/collections/naip/items/flood-imagery-2016-ortho
ID: tippecanoe-2013-ortho, URL: https://stac.d2s.org/collections/naip/items/tippecanoe-2013-ortho
ID: purdue-2013-ortho, URL: https://stac.d2s.org/collections/naip/items/purdue-2013-ortho


You can see from the results above that there are 5 orthomosaic images are available. We will use the latest one.

In [16]:
# Choose the first item
item = items[0]

# Print the name of the asset available in the STAC item
for asset in item.assets:
    print(asset)

#The URL for the NAIP raster is in the "image" asset
ortho_url = item.assets["ortho-image"].href
print(ortho_url)

ortho-image
https://lidar.digitalforestry.org/state/2023/tippecanoe/tippecanoe_2023_ortho.tif


Now, let's clip it using our bounding box.

In [17]:
# Desired location and file name for the clipped raster
clipped_ortho_fn = 'ortho_data/clipped_ortho.tif'

# Need to turn the bbox into GeoJSON object
bbox_geojson = bbox_to_geojson(bbox)

# Now let's clip it
clip_by_mask(in_raster=ortho_url, geojson=bbox_geojson, out_raster=clipped_ortho_fn)

Let's visualize orthomosaic and building footprint together.

In [18]:
m = leafmap.Map()
m.add_raster(clipped_ortho_fn, layer_name="Indiana Ortho 2023")
m.add_geojson(buildings_geojson, layer_name="Buildings")
m

Map(center=[40.42775, -86.914749], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

We can still see that Indiana Orthomosaic image is not perfectly rectified. Let's check this with Indiana LiDAR data layer.

In [19]:
# Search for items from the Indiana Ortho Imagery Collection
search = client.search(
    max_items=10,
    collections=["ndhm"],
    bbox=bbox,
    datetime=['2011-01-01', '2024-12-31'],
)

# Print STAC Item ID and STAC Browser URL for search results
stac_browser_base_item_url = "https://stac.d2s.org/collections/naip/items"
items = []
for item in search.items():
    print(f"ID: {item.id}, URL: {stac_browser_base_item_url}/{item.id}")
    items.append(item)

# Choose the first item
item = items[0]

# Print the name of the asset available in the STAC item
for asset in item.assets:
    print(asset)

#The URL for the NAIP raster is in the "image" asset
ndhm_url = item.assets["lidar-image"].href
print(ndhm_url)

# Desired location and file name for the clipped raster
clipped_ndhm_fn = 'lidar_data/clipped_ndhm.tif'

# Need to turn the bbox into GeoJSON object
bbox_geojson = bbox_to_geojson(bbox)

# Now let's clip it
clip_by_mask(in_raster=ndhm_url, geojson=bbox_geojson, out_raster=clipped_ndhm_fn)

ID: tippecanoe-ndhm, URL: https://stac.d2s.org/collections/naip/items/tippecanoe-ndhm
lidar-image
https://lidar.digitalforestry.org/QL2_3DEP_LiDAR_IN_2017_2019_l2/tippecanoe/cog/tippecanoe_ndhm_2018.tif


Now, let's add all the layers together and compare.

In [20]:
building_footprint_style = {
    "stroke": True,
    "color": "#ffffff",
    "weight": 1.5,
    "opacity": 1,
    "fill": True,
    "fillColor": "#0000ff",
    "fillOpacity": 0.1,
}

m = leafmap.Map()
m.add_raster(clipped_naip_fn, layer_name="NAIP")
m.add_raster(clipped_ortho_fn, layer_name="Indiana Ortho 2023")
m.add_raster(clipped_ndhm_fn, colormap="jet", vmin=0, vmax=80,layer_name="LiDAR NDHM 2018")
m.add_geojson(buildings_geojson,style=building_footprint_style, layer_name="Buildings")
m

Map(center=[40.427754, -86.914749], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…